# Multivariate CNN Models - Multiple Input Series (Multi-headed)

In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate

Using TensorFlow backend.


#### split a univariate sequence into samples

In [2]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

## 1. Data preparation

#### define input sequence

In [3]:
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

#### convert to [rows, columns] structure

In [4]:
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

#### horizontally stack column

In [5]:
dataset = np.hstack((in_seq1, in_seq2, out_seq))

In [6]:
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

#### choose a number of time steps

In [7]:
n_steps = 3

#### convert into input/output

In [8]:
X, y = split_sequences(dataset, n_steps)

#### one time series per head

In [9]:
n_features = 1

#### separate input data

In [10]:
X1 = X[:, :, 0].reshape(X.shape[0], X.shape[1], n_features)
X2 = X[:, :, 1].reshape(X.shape[0], X.shape[1], n_features)

In [11]:
X1

array([[[10],
        [20],
        [30]],

       [[20],
        [30],
        [40]],

       [[30],
        [40],
        [50]],

       [[40],
        [50],
        [60]],

       [[50],
        [60],
        [70]],

       [[60],
        [70],
        [80]],

       [[70],
        [80],
        [90]]])

In [12]:
X2

array([[[15],
        [25],
        [35]],

       [[25],
        [35],
        [45]],

       [[35],
        [45],
        [55]],

       [[45],
        [55],
        [65]],

       [[55],
        [65],
        [75]],

       [[65],
        [75],
        [85]],

       [[75],
        [85],
        [95]]])

## 2. CNN Model

#### define model

#### first input model

In [13]:
visible1 = Input(shape=(n_steps, n_features))
cnn1 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible1)
cnn1 = MaxPooling1D(pool_size=2)(cnn1)
cnn1 = Flatten()(cnn1)

#### second input model

In [14]:
visible2 = Input(shape=(n_steps, n_features))
cnn2 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible2)
cnn2 = MaxPooling1D(pool_size=2)(cnn2)
cnn2 = Flatten()(cnn2)

#### merge input models

In [15]:
merge = concatenate([cnn1, cnn2])
dense = Dense(50, activation='relu')(merge)
output = Dense(1)(dense)
model = Model(inputs=[visible1, visible2], outputs=output)
model.compile(optimizer='adam', loss='mse')

#### fit model

In [16]:
model.fit([X1, X2], y, epochs=1000, verbose=0)

#### demonstrate prediction

In [17]:
x_input = np.array([[80, 85], [90, 95], [100, 105]])
x1 = x_input[:, 0].reshape((1, n_steps, n_features))
x2 = x_input[:, 1].reshape((1, n_steps, n_features))

In [18]:
x1

array([[[ 80],
        [ 90],
        [100]]])

In [19]:
x2

array([[[ 85],
        [ 95],
        [105]]])

In [20]:
yhat = model.predict([x1, x2], verbose=0)
print(yhat)

[[205.65668]]
